## Notbook for calculating the Global Wind Atlas parts to be downloaded, conforming with the 250.000 km**2 restrictions

In Step 1: Shapes are read in, split into equal parts and exported as .shp files. These can be uploded to globalwindatlas.info/. Files should be placed in ROOT + 'data/gwa/'

In Step 2: The downloaded GWA.tif rasters for one country are stiched together.

In [1]:
import math
from shapely.geometry import LineString, MultiPolygon, Polygon, box
from shapely.ops import split

import geopandas as gpd
import xarray as xr
import rioxarray as rioxr 
import rasterio as rio

from utils import geo, data

import cartopy.io.shapereader as shpreader

import yaml

from rioxarray.merge import merge_arrays

In [122]:
ROOT = 'C:/Users/49176/Uni/Masterthesis/'
LUISA = ROOT + 'data/input_geodata/luisa.tif'

### 0. Define functions

In [27]:
def gwa_bounds_prep(country, padding, shape_filter, savepath = ROOT + 'data/country_gwa_bounds/'):
    
    #construct bbox df with adjusted bounds for full era5 overlap
    bounds = country.to_crs(3035).bounds.values[0]
    bb = box(bounds[0]-padding, bounds[1]-padding, bounds[2]+padding, bounds[3]+padding)
    bb_df = gpd.GeoDataFrame(gpd.GeoSeries(bb), columns=['geometry'], crs = 3035)
    bb_df["area"] = bb_df['geometry'].area/ 10**6
    
    #identify needed parts according to GWA 250000 km**2 limitation
    parts = math.ceil(bb_df['area']/250000)
    
    if parts == 1:
        bb_df.to_crs(4326).to_file(savepath + f'{cntry}.shp')
        return parts
    else:
        minx, miny, maxx, maxy = bb.bounds
        
        parts = parts * 2

        dx = (maxx - minx) / parts  # width of a small part

        vertical_splitters = [LineString([(minx + i*dx, miny), (minx + i*dx, maxy)]) for i in range(parts)]
                      
        bb_split = bb
        for splitter in vertical_splitters:
            bb_split = MultiPolygon(split(bb_split, splitter))
                      
        for i in range(len(bb_split.geoms)):
            sp = savepath + f'{cntry}_part_{str(i+1)}.shp'
            gpd.GeoDataFrame([bb_split.geoms[i]], columns=['geometry'], crs = 3035).to_crs(4326).to_file(sp)
            
        return parts
            

In [16]:
def gwa_bounds_prep(country, padding, shape_filter, savepath = ROOT + 'data/country_gwa_bounds/'):
    
    #construct bbox df with adjusted bounds for full era5 overlap
    bounds = country.to_crs(3035).bounds.values[0]
    bb = box(bounds[0]-padding, bounds[1]-padding, bounds[2]+padding, bounds[3]+padding)
    bb_df = gpd.GeoDataFrame(gpd.GeoSeries(bb), columns=['geometry'], crs = 3035)
    bb_df["area"] = bb_df['geometry'].area/ 10**6
    
    #identify needed parts according to GWA 250000 km**2 limitation
    parts = math.ceil(bb_df['area']/250000)
    
    if parts == 1:
        bb_df.to_crs(4326).to_file(savepath + f'{cntry}.shp')
        return parts
    else:
        
        ny = parts #math.floor(parts/2)
        nx = parts # .ceil(parts/2)

        minx, miny, maxx, maxy = bb.bounds
        dx = (maxx - minx) / nx  # width of a small part
        dy = (maxy - miny) / ny  # height of a small part
        horizontal_splitters = [LineString([(minx, miny + i*dy), (maxx, miny + i*dy)]) for i in range(ny)]
        vertical_splitters = [LineString([(minx + i*dx, miny), (minx + i*dx, maxy)]) for i in range(nx)]
        splitters = horizontal_splitters + vertical_splitters
        bb_split = bb

        for splitter in splitters:
            bb_split = MultiPolygon(split(bb_split, splitter))
            
        for i in range(len(bb_split.geoms)):
            sp = savepath + f'{cntry}_part_{str(i+1)}.shp'
            gpd.GeoDataFrame([bb_split.geoms[i]], columns=['geometry'], crs = 3035).to_crs(4326).to_file(sp)
            
        return parts**2
            

### Step 1

In [17]:
with open("countrycode.yaml", 'r') as stream:
    countrycode = yaml.safe_load(stream)  
    
countries = countrycode.keys()

In [24]:
part_d = {}

for cntry in countries:
    country = gpd.read_file(ROOT + f'data/country_shapes/{cntry}.gpkg')
    part_d[cntry] =  gwa_bounds_prep(country, padding=30000, shape_filter = LUISA)

In [25]:
# Save shapecount of country
part_d2plus = {k:v for k,v in zip(part_d.keys(), part_d.values()) if not v == 1}

### Step 2

In [ ]:
for cntry in countries:
    
    ls = []    
    folder = os.listdir(ROOT + f'data/gwa/')
    country_files = [file for file in folder if cntry.lower() in file]
    
    print(country_files)
    
    for i in country_files:
    
        with rioxr.open_rasterio(ROOT + f'data/gwa/{i}') as f:
            gwa = f
            f.close()
        ls.append(gwa)
        
    country_gwa = merge_arrays(ls, nodata = 0)
    country_gwa.rio.to_raster(ROOT + f'data/gwa/{cntry}_wind-speed_100m.tif', recalc_transform = True)